In [1]:
from src.load_data import *
from src.metrics import *
from model.model import *

Using TensorFlow backend.


MNIST dataset loaded, scaled but not flattened, available as x and y.


### Baseline

In [2]:
kmeans = KMeans(n_clusters=10, n_jobs=4)
y_pred_kmeans = kmeans.fit_predict(np.reshape(x, [-1, 28*28]))
print("Baseline accuracy is {:.2%}." format(acc(y, y_pred_kmeans)))

0.14922857142857143

### Custom Autoencoder

In [3]:
autoencoder, encoder = model(hidlayer_size = 150, codings_size = 60,
                                                  batch_size = 1000, alpha = 0.2, lamda = 0.01)
encoder.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 784)          0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 150)          117750      flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 60)           9060        dense_1[0][0]                    
___________

In [6]:
autoencoder.fit(x, x, batch_size = 2000, shuffle = True, epochs = 20)
_, _, hid = ncoder.predict(x, batch_size = 1000)

In [7]:
y_pretrained_kmeans = kmeans.fit_predict(hid)
print("Baseline accuracy is {:.2%}." format(acc(y, y_pretrained_kmeans)))

0.2143

In [ ]:
plot_score(hid, y)